In [1]:
from sys import path
from os.path import relpath, realpath, abspath, join, exists
from itertools import product

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import h5py

from functools import partial

pd.set_option('display.max_rows', 999)

In [2]:
from torchnetjson.builder import build_net

In [3]:
from thesis_v2 import dir_dict
from thesis_v2.data import load_data_helper
from thesis_v2.training.training_aux import load_training_results
from thesis_v2.training_extra.misc import count_params
from thesis_v2.models.maskcnn_polished_with_local_pcn.builder import load_modules

load_modules()

In [4]:
folder_to_check = 'scripts/training/yuanyuan_8k_a_3day/maskcnn_polished_with_local_pcn'
path.insert(0, join(dir_dict['root'], folder_to_check))
from submit_certain_configs import param_iterator_obj
from key_utils import keygen

In [5]:
# compute ccmax
from strflab.stats import cc_max
from thesis_v2.data.prepared.yuanyuan_8k import get_neural_data_per_trial

cc_max_all_neurons = cc_max(get_neural_data_per_trial(
    ('042318', '043018','051018',))
                           )
assert cc_max_all_neurons.shape == (79,)

In [6]:
from thesis_v2.training_extra.data import generate_datasets
from thesis_v2.training_extra.evaluation import eval_fn_wrapper as eval_fn_wrapper_neural
from thesis_v2.training.training import eval_wrapper
from thesis_v2.data.prepared.yuanyuan_8k import get_data, get_neural_data

In [7]:
from torch.backends import cudnn
import torch
cudnn.enabled = True
cudnn.deterministic = True
cudnn.benchmark = False

_data_cache = dict()

def get_data_cached(input_size, seed):
    if (input_size, seed) not in _data_cache:
        _data_cache[input_size, seed] = get_data('a', 200, input_size,
                                                 ('042318', '043018', '051018'),
                                                 scale=0.5,
                                                 seed=seed)
    return _data_cache[input_size, seed]

In [8]:
def plot_responses(responses_3d):
    # time x num_image x num_neuron
    
    n_time, n_image, n_neuron = responses_3d.shape
    assert n_time == 6 and n_image == 1600 and n_neuron == 79
    for idx_neuron in range(n_neuron):
        responses_3d_this = responses_3d[:,:,idx_neuron]
        responses_mean = responses_3d_this.mean(axis=1)
        responses_std = responses_3d_this.std(axis=1)
        print(idx_neuron, responses_mean, responses_std)

In [13]:
def load_everything():
    _df_collect = []
    for idx, param in enumerate(param_iterator_obj.generate()):
        if idx % 50 == 0:
            print(idx)
            
#         print(len(param))
        
        assert len(param) == 23
        assert param['split_seed'] == 'legacy'
        assert param['out_channel'] == 16
        assert param['num_layer'] == 2
        assert param['kernel_size_l1'] == 9
        assert param['pooling_ksize'] == 3
        assert param['pooling_type'] == 'avg'
        
        if param['pcn_cls'] != 5:
            continue
        
#         assert param['model_seed'] == 0
        
        key = keygen(**{k: v for k, v in param.items() if k not in {'scale', 'smoothness'}})
        # 10 to go.
        result = load_training_results(key, return_model=False)
        # load twice, first time to get the model.
        result = load_training_results(key, return_model=True, model=build_net(result['config_extra']['model']))
        
        
        
        datasets = get_data_cached(param['input_size'], param['split_seed'])
        
        datasets = {
            'X_train': datasets[0].astype(np.float32),
            'y_train': datasets[1],
            'X_val': datasets[2].astype(np.float32),
            'y_val': datasets[3],
            'X_test': datasets[4].astype(np.float32),
            'y_test': datasets[5],
        }
        
        # only the test one is needed.
        datasets_all = generate_datasets(
            **datasets,
            per_epoch_train=True, shuffle_train=True,
        )
        
        datasets_test = datasets_all['test']
        datasets_train = datasets_all['train']
        
        responses_test_all = []
        
        for c in range(6):
            
            # hack models's cycle
            result['model'].moduledict['conv1'].num_cycle = c
            result_on_the_go = eval_wrapper(result['model'].cuda(),
                                            datasets_test,
                                            'cuda',
                                            1,
                                            partial(eval_fn_wrapper_neural, loss_type=param['loss_type']),
                                            (lambda dummy1,dummy2,dummy3: torch.tensor(0.0)),
                                            return_responses=True
                                           )
        
            responses_test = np.concatenate([x[0] for x in result_on_the_go['responses']], axis=0)
            print(responses_test.shape)
            responses_test_all.append(responses_test)

        plot_responses(np.asarray(responses_test_all))
        
        
        
        
        num_epochs = [len(x) for x in result['stats_all']]
        
        cc_raw = np.asarray(result['stats_best']['stats']['test']['corr'])
        assert cc_raw.shape == (79,)
        
        _df_collect.append(
            {
#                 'split_seed': str(param['split_seed']),
                
                
                'act_fn': param['act_fn'],
                'bn_before_act': param['bn_before_act'],
                'bn_after_fc': param['bn_after_fc'],
                'input_size': param['input_size'],
                'loss_type': param['loss_type'],
                
                # 12 to go
                
                'scale': float(param['scale']),
                # 
                'smoothness': float(param['smoothness']),
                # 
                
                # 8 to go
                'pcn_bn': param['pcn_bn'],
                'pcn_bn_post': param['pcn_bn_post'],
                'pcn_bypass': param['pcn_bypass'],
                'pcn_final_act': param['pcn_final_act'],
                'pcn_no_act': param['pcn_no_act'],
                'pcn_bias': param['pcn_bias'],
                
                'pcn_cls': param['pcn_cls'],
                
                'model_seed': param['model_seed'],
                
#                 'corr_test': result['stats_best']['stats']['test']['corr_mean'],
                'corr_test': ((cc_raw/cc_max_all_neurons)**2).mean(),
                'max_epoch': max(num_epochs),
                'num_param': count_params(result['model']),
            }
        )
    _df =  pd.DataFrame(_df_collect, columns=[
#             'split_seed',
        'act_fn', 'bn_before_act', 'bn_after_fc',
        'input_size', 'loss_type', 'scale',
        'smoothness',

        'pcn_bn', 'pcn_bn_post', 'pcn_bypass', 'pcn_final_act', 'pcn_no_act', 'pcn_bias', 'pcn_cls',
        'model_seed',

        'corr_test', 'num_param',
        'max_epoch',

                                             ])
    _df = _df.set_index([
        'act_fn', 'bn_before_act', 'bn_after_fc',
        'input_size', 'loss_type', 'scale',
        'smoothness',
        'pcn_bn', 'pcn_bn_post', 'pcn_bypass', 'pcn_final_act', 'pcn_no_act', 'pcn_bias', 'pcn_cls',
        'model_seed',
    ],verify_integrity=True)
    return _df

In [14]:
# the intermediate responses can be increasing/decreasing/staying the same, etc. all sorts of things.

# I guess that's expected given these intermediate responses are not trained to provide good prediction.

# compare this with `results_processed/yuanyuan_8k_a_3day/maskcnn_polished_with_rcnn_k_bl/temporal_response_for_20200218.ipynb`,
# which are trained models and thus stay the same.

df = load_everything()

0
(1600, 79)
(1600, 79)
(1600, 79)
(1600, 79)
(1600, 79)
(1600, 79)
0 [0.30198482 0.33236665 0.36917773 0.4057083  0.44731864 0.49295714] [0.04624632 0.07034678 0.07831167 0.08965635 0.10759974 0.13128527]
1 [0.5255639  0.533459   0.53099805 0.52166224 0.5085861  0.49565428] [0.03887381 0.05862324 0.0739524  0.09316805 0.11702812 0.14497142]
2 [0.4921613  0.5188626  0.52550894 0.52401984 0.5142248  0.49744713] [0.02718608 0.05595719 0.0789962  0.09898416 0.11832145 0.13734345]
3 [0.610166   0.60584205 0.59316844 0.5741143  0.5442244  0.5008847 ] [0.00412216 0.01419461 0.03384415 0.05339107 0.074862   0.09984972]
4 [0.6124203  0.6110943  0.59466463 0.5680705  0.5334105  0.4988688 ] [0.04924123 0.0891249  0.10658973 0.12280478 0.14460157 0.1691524 ]
5 [0.16912684 0.24209347 0.302116   0.36244556 0.431771   0.5026544 ] [0.02326577 0.04758193 0.0695722  0.08952046 0.10717238 0.12112662]
6 [0.58982617 0.5701758  0.5519113  0.5305136  0.5126352  0.495467  ] [0.05590381 0.1203348  0.14956398 

(1600, 79)
(1600, 79)
(1600, 79)
(1600, 79)
0 [0.30909362 0.3350302  0.36119202 0.39790207 0.44508857 0.501028  ] [0.00481698 0.03813183 0.0644873  0.09022187 0.11476023 0.13666515]
1 [0.26824245 0.32915998 0.36547226 0.40289146 0.44651946 0.49729043] [0.00294494 0.04159614 0.06835894 0.09369802 0.12030026 0.14912109]
2 [0.49058333 0.50862426 0.5083491  0.506078   0.502763   0.49842942] [0.01330448 0.04464008 0.06911419 0.09267373 0.11530688 0.13699464]
3 [0.5551219  0.5478916  0.5391014  0.5284671  0.515462   0.49988282] [0.02094814 0.0314135  0.04448742 0.06057989 0.07878927 0.09861147]
4 [0.46409363 0.48412517 0.48467612 0.48708138 0.4927503  0.50087786] [0.01021294 0.05539133 0.08695897 0.11532268 0.14170973 0.16587009]
5 [0.29410815 0.34084588 0.37187997 0.40808648 0.45113045 0.5006935 ] [0.00664731 0.03858948 0.06077968 0.08257577 0.10446066 0.12587303]
6 [0.41394833 0.4282319  0.44036254 0.45605573 0.4752508  0.49820614] [0.03620955 0.0727288  0.1011153  0.12834746 0.15676245 0.

(1600, 79)
(1600, 79)
(1600, 79)
(1600, 79)
0 [0.41227654 0.44200093 0.45326573 0.46459767 0.47815925 0.4949028 ] [0.01891639 0.05769695 0.08099838 0.10255774 0.12156425 0.13813794]
1 [0.6405327  0.58694494 0.56044304 0.5316096  0.5065029  0.49567208] [0.01194958 0.06278189 0.09327812 0.11993425 0.1385375  0.14981924]
2 [0.46820426 0.5205427  0.5237377  0.5177769  0.50812757 0.4961537 ] [0.0232262  0.08410624 0.10280297 0.11684483 0.12826146 0.13818155]
3 [0.5558128  0.57037073 0.5654148  0.5502347  0.52695334 0.49785054] [0.00991389 0.03242824 0.04970911 0.06672284 0.08408798 0.10103963]
4 [0.81494814 0.69400024 0.6273907  0.5666707  0.5218821  0.49830133] [0.01726329 0.06070022 0.10225698 0.13558654 0.15491158 0.16899331]
5 [0.40654495 0.46446404 0.4711211  0.47564888 0.48403177 0.49763462] [0.01542696 0.07454008 0.09206341 0.10546355 0.11617972 0.12531962]
6 [0.32409495 0.37531945 0.41168374 0.442594   0.47013947 0.4953567 ] [0.08651889 0.14382784 0.17209156 0.18939193 0.19977137 0.

(1600, 79)
(1600, 79)
(1600, 79)
(1600, 79)
0 [0.41030502 0.45799533 0.47772416 0.49009573 0.49691713 0.4997256 ] [0.00591015 0.04725659 0.07463596 0.099536   0.12038563 0.13814758]
1 [0.50762534 0.55915445 0.5631657  0.55023575 0.5277012  0.49705863] [0.01157297 0.07400625 0.09467454 0.11413861 0.13235182 0.14965095]
2 [0.43416122 0.47667608 0.49218678 0.50062346 0.5026199  0.4988425 ] [0.0088496  0.04401543 0.06797197 0.09276378 0.11687732 0.14095756]
3 [0.8074242  0.74861807 0.6937839  0.6348488  0.56961733 0.49709892] [0.00126795 0.01886064 0.03828409 0.05745575 0.07798601 0.1009183 ]
4 [0.5152391  0.56019753 0.56718576 0.5588052  0.53664047 0.50153834] [0.01367275 0.0787633  0.11230408 0.14108446 0.16272901 0.1782494 ]
5 [0.3631761  0.41912043 0.4482366  0.46863997 0.48507637 0.49991584] [0.01298853 0.03509956 0.05321978 0.07306618 0.09629684 0.12440587]
6 [0.40633476 0.43205544 0.44498643 0.45982802 0.47555825 0.49315557] [0.01505324 0.04883234 0.07964326 0.11317764 0.15158223 0.

(1600, 79)
(1600, 79)
(1600, 79)
(1600, 79)
0 [0.48161483 0.4832362  0.48557281 0.48846868 0.4928688  0.5017013 ] [0.00331473 0.00842537 0.01690348 0.03336595 0.06818686 0.14182827]
1 [0.43087867 0.4322132  0.43319243 0.43623757 0.44878495 0.4948053 ] [0.00192518 0.00545557 0.01283948 0.03066619 0.07167619 0.16397084]
2 [0.46582925 0.46940765 0.4733075  0.47810715 0.48624855 0.5063362 ] [0.00344919 0.00861055 0.01751688 0.03445673 0.07058189 0.14995906]
3 [0.5385595  0.537586   0.535215   0.52954566 0.51816636 0.49899793] [0.00149822 0.00385031 0.00830949 0.0194204  0.04615839 0.10552055]
4 [0.47857055 0.48184806 0.4843987  0.48591682 0.4892885  0.50611687] [0.0043804  0.01004979 0.01956874 0.03965135 0.08396611 0.1821417 ]
5 [0.3778967  0.3817836  0.3886114  0.40199432 0.4300232  0.49167168] [0.003813   0.00939574 0.01765353 0.03271193 0.06376596 0.13007712]
6 [0.4393555  0.43877247 0.43873733 0.43999198 0.44721925 0.47909224] [0.00442678 0.00987253 0.01919597 0.03928165 0.0866503  0.

...

(1600, 79)
(1600, 79)
(1600, 79)
(1600, 79)
0 [0.5203184  0.51867163 0.5144029  0.50544304 0.49378228 0.4827982 ] [0.00178239 0.00675432 0.0174615  0.03518618 0.06541562 0.12427913]
1 [0.4844627  0.4860322  0.4898066  0.48914704 0.48543417 0.50312555] [0.00202245 0.00553034 0.01292371 0.03315033 0.07756373 0.16817392]
2 [0.48755485 0.48924232 0.48745766 0.47739154 0.47147056 0.50418496] [0.00296884 0.00981679 0.02145251 0.03913847 0.07228751 0.14794102]
3 [0.52484304 0.5293362  0.52687925 0.5172575  0.5033847  0.5012107 ] [0.0021142  0.00633386 0.01402071 0.02933156 0.0568348  0.09931663]
4 [0.49615175 0.49668378 0.49330643 0.48570228 0.47925907 0.48808014] [0.00214313 0.00765855 0.01762435 0.03511699 0.06934837 0.14031944]
5 [0.4745746  0.4807367  0.4898536  0.4879139  0.4800738  0.49261597] [0.00148494 0.00420955 0.01318759 0.03100799 0.06473908 0.1303165 ]
6 [0.49095932 0.507137   0.52544403 0.52648276 0.50534934 0.48511413] [0.00318831 0.01449745 0.0387537  0.07254283 0.12014174 0.

(1600, 79)
(1600, 79)
(1600, 79)
(1600, 79)
0 [0.74332327 0.74856675 0.6921155  0.6173668  0.54866374 0.4955612 ] [0.01214697 0.03394233 0.05576082 0.0736936  0.09925567 0.12995613]
1 [0.5802585  0.61302453 0.62066793 0.587192   0.53695714 0.49686638] [0.01289016 0.03493594 0.06401619 0.09060005 0.11659602 0.1453315 ]
2 [0.92648333 0.8673985  0.77089936 0.6623015  0.5673047  0.4954677 ] [0.011264   0.02715196 0.0500565  0.07528222 0.10720558 0.14017884]
3 [0.45722666 0.5604732  0.57394487 0.5586314  0.5262082  0.49930748] [0.02747576 0.01997617 0.02345615 0.04150689 0.07519336 0.10651714]
4 [0.7870099  0.81089234 0.77038497 0.683049   0.5822675  0.49900588] [0.01302198 0.03375598 0.06530318 0.09798543 0.1313452  0.166673  ]
5 [0.84271544 0.7824614  0.6992697  0.6126947  0.5450068  0.5012651 ] [0.01984364 0.04444803 0.07171929 0.08663682 0.10219442 0.12585242]
6 [0.38386112 0.409299   0.37707275 0.36706492 0.40056953 0.4849245 ] [0.01932974 0.05490726 0.06830992 0.08596771 0.12463263 0.

(1600, 79)
(1600, 79)
(1600, 79)
(1600, 79)
0 [0.371504   0.36618564 0.35987756 0.37048736 0.41173038 0.4940796 ] [0.0060686  0.01299605 0.01825124 0.03317828 0.06960488 0.12867855]
1 [0.28141108 0.27743968 0.27837643 0.30091122 0.36690694 0.4942581 ] [0.00187149 0.00419564 0.00985418 0.03060938 0.07736567 0.1484692 ]
2 [0.4943657  0.4659579  0.44015893 0.42913222 0.44753098 0.4963422 ] [0.00341186 0.01070012 0.02155025 0.04510386 0.0872163  0.13773187]
3 [0.506769   0.498542   0.49396133 0.49094954 0.49166188 0.49837667] [0.00560368 0.01174699 0.02115115 0.0403328  0.06829647 0.09760863]
4 [0.31762227 0.31468582 0.31437814 0.33324477 0.38939536 0.49801958] [0.0043385  0.00890744 0.01626818 0.04078773 0.09143555 0.16429114]
5 [0.45737883 0.4355732  0.41269165 0.40656132 0.4311564  0.4972406 ] [0.00487099 0.00976642 0.01671392 0.03401483 0.06988794 0.12384455]
6 [0.4545295  0.47363603 0.4883841  0.48918855 0.48114738 0.4798493 ] [0.01173883 0.02488256 0.04507602 0.07827436 0.12835611 0.

(1600, 79)
(1600, 79)
(1600, 79)
(1600, 79)
(1600, 79)
(1600, 79)
0 [0.41669482 0.42075974 0.4287518  0.44093463 0.46075082 0.50644577] [0.00056176 0.00360041 0.01166547 0.02573412 0.05558744 0.12785529]
1 [0.43725547 0.44835576 0.47059035 0.4924265  0.49588966 0.49202386] [0.00242215 0.00770675 0.02669792 0.05318572 0.08663804 0.14936382]
2 [0.44479883 0.44621155 0.4590307  0.47695404 0.49115217 0.5055603 ] [0.00133527 0.00489379 0.01587187 0.03313623 0.06777897 0.140003  ]
3 [0.61541057 0.615273   0.6142086  0.6082332  0.5761411  0.49352616] [1.07460364e-04 6.61901489e-04 1.93898298e-03 8.28799233e-03
 4.25852500e-02 1.07860930e-01]
4 [0.39767227 0.40469474 0.4171405  0.43396682 0.4578024  0.51064473] [0.00045047 0.00352799 0.01128142 0.02643991 0.06659973 0.16207995]
5 [0.35738525 0.36470968 0.379934   0.4044319  0.44179565 0.5142222 ] [0.00066282 0.0035984  0.01176135 0.0247113  0.05003014 0.1184685 ]
6 [0.50438285 0.4922438  0.5101171  0.53134894 0.52446723 0.49287936] [0.00652547

KeyboardInterrupt: 